<a href="https://colab.research.google.com/github/zooodung/TA-Prediction/blob/woogie/onehotencoding_RandomForest_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
path='/content/drive/MyDrive/camp/project1/open/'

In [4]:
daegu_data=pd.read_csv('/content/drive/MyDrive/open/open/train.csv')
wide_data=pd.read_csv('/content/drive/MyDrive/open/open/external_open/countrywide_accident.csv')
test_data=pd.read_csv('/content/drive/MyDrive/open/open/test.csv')
daegu_data.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


In [5]:
columns_to_drop = ['ID', '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령',
                   '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']

daegu=daegu_data.drop(columns=columns_to_drop)



In [6]:
daegu[['시','구','동']] = daegu['시군구'].str.split(' ',expand=True)
daegu.drop(['시군구'],inplace=True, axis=1)

In [7]:
daegu[['도로형태1', '도로형태2']] = daegu['도로형태'].str.split(' - ', expand=True)
daegu.drop(['도로형태'],inplace=True, axis=1)

In [8]:
daegu.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타


In [9]:
wide=wide_data.copy()
len(wide)

602775

In [10]:
wide = wide[~wide['도로형태'].str.split().str[-1].str.contains('철길건널목')]
len(wide)

602763

In [11]:
wide=wide.drop(columns=columns_to_drop)

In [12]:
def contains_specific_location(address):
    cities = ['서울특별시', '인천광역시', '부산광역시', '울산광역시', '광주광역시', '대전광역시']
    for city in cities:
        if city in address:
            return True
    return False
wide=wide[wide['시군구'].apply(contains_specific_location)]

In [13]:
wide['시군구'].apply(lambda x:x.split()[0]).value_counts()

서울특별시    108395
부산광역시     36244
인천광역시     25115
광주광역시     23430
대전광역시     22579
울산광역시     11981
Name: 시군구, dtype: int64

In [14]:
wide.drop(['시군구'],inplace=True, axis=1)

In [16]:
wide[['도로형태1', '도로형태2']] = wide['도로형태'].str.split(' - ', expand=True)
wide.drop(['도로형태'],inplace=True, axis=1)

In [17]:
wide.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,교차로,교차로횡단보도내
4,2019-01-01 00,화요일,맑음,건조,차대차,0,0,2,1,7,교차로,교차로안
5,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,교차로,교차로부근
8,2019-01-01 00,화요일,맑음,건조,차대차,0,0,1,0,3,단일로,기타
11,2019-01-01 00,화요일,맑음,건조,차대사람,1,0,0,0,10,교차로,교차로안


데이터 합병

In [18]:
print(len(daegu),len(wide), len(daegu)+len(wide))

39609 227744 267353


In [19]:
concatenated_df = pd.concat([daegu, wide])
concatenated_df

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,2021-12-31 23,금요일,맑음,건조,차대사람,0,0,1,0,3,NaN,NaN,NaN,단일로,기타
602769,2021-12-31 23,금요일,맑음,건조,차대차,0,1,0,0,5,NaN,NaN,NaN,단일로,기타
602770,2021-12-31 23,금요일,흐림,건조,차대차,0,1,1,0,8,NaN,NaN,NaN,교차로,교차로안
602771,2021-12-31 23,금요일,맑음,건조,차대차,0,0,2,0,6,NaN,NaN,NaN,단일로,기타


In [20]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [21]:
concatenated_df['사고일시'] = pd.to_datetime(concatenated_df['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
concatenated_df['년도'] = concatenated_df['사고일시'].dt.year
concatenated_df['월'] = concatenated_df['사고일시'].dt.month
concatenated_df['day'] = concatenated_df['사고일시'].dt.day
concatenated_df['시간'] = concatenated_df['사고일시'].dt.hour
concatenated_df['day_of_week'] = concatenated_df['사고일시'].dt.dayofweek


concatenated_df['holiday'] = np.where((concatenated_df.day_of_week >= 5) | (concatenated_df.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')


concatenated_df.drop(columns=['사고일시'], inplace=True)


concatenated_df['holiday'].value_counts()

평일    195393
휴일     71960
Name: holiday, dtype: int64

In [22]:
concatenated_df.drop(['day', 'day_of_week'], inplace=True, axis=1)

In [23]:
def seperate_time(time): #심야, 새벽, 오전, 점심, 오후, 저녁, 야간 ->4시간단위
    if 0<=int(time)<4:
        return '심야'
    elif int(time)<8:
        return '새벽'
    elif int(time)<12:
        return '오전'
    elif int(time)<16:
        return '오후'
    elif int(time)<20:
        return '저녁'
    else:
        return '야간'
concatenated_df['시간구분']=concatenated_df['시간'].apply(seperate_time)
print(concatenated_df['시간구분'].value_counts().sum())
concatenated_df['시간구분'].value_counts()

267353


저녁    71768
오후    58743
오전    50058
야간    47737
새벽    20554
심야    18493
Name: 시간구분, dtype: int64

In [24]:
def seperate_month(month):
    if int(month) in [12,1,2]:
        return '겨울'
    elif int(month) in [3,4,5]:
        return '봄'
    elif int(month) in [6,7,8]:
        return '여름'
    else:
        return '가을'
concatenated_df['계절']=concatenated_df['월'].apply(seperate_month)
print(concatenated_df['계절'].value_counts().sum())
concatenated_df['계절'].value_counts()

267353


가을    70234
여름    68434
봄     66607
겨울    62078
Name: 계절, dtype: int64

In [25]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타,2019,1,0,휴일,심야,겨울
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타,2019,1,0,휴일,심야,겨울
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타,2019,1,1,휴일,심야,겨울
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타,2019,1,2,휴일,심야,겨울
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타,2019,1,4,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,NaN,NaN,NaN,단일로,기타,2021,12,23,평일,야간,겨울
602769,금요일,맑음,건조,차대차,0,1,0,0,5,NaN,NaN,NaN,단일로,기타,2021,12,23,평일,야간,겨울
602770,금요일,흐림,건조,차대차,0,1,1,0,8,NaN,NaN,NaN,교차로,교차로안,2021,12,23,평일,야간,겨울
602771,금요일,맑음,건조,차대차,0,0,2,0,6,NaN,NaN,NaN,단일로,기타,2021,12,23,평일,야간,겨울


#test data


In [26]:
test=test_data.copy()

In [27]:
test[['시','구','동']] = test['시군구'].str.split(' ',expand=True)
test.drop(['시군구'],inplace=True, axis=1)

In [28]:
test[['도로형태1', '도로형태2']] = test['도로형태'].str.split(' - ', expand=True)
test.drop(['도로형태'],inplace=True, axis=1)

In [29]:
test['사고일시'] = pd.to_datetime(test['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
test['년도'] = test['사고일시'].dt.year
test['월'] = test['사고일시'].dt.month
test['day'] = test['사고일시'].dt.day
test['시간'] = test['사고일시'].dt.hour
test['day_of_week'] = test['사고일시'].dt.dayofweek

test['holiday'] = np.where((test.day_of_week >= 5) | (test.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

test.drop(columns=['사고일시'], inplace=True)
test.drop(['day', 'day_of_week'], inplace=True, axis=1)

In [30]:
test['시간구분']=test['시간'].apply(seperate_time)
print(test['시간구분'].value_counts().sum())
test['시간구분'].value_counts()

10963


저녁    3168
오후    2548
오전    2196
야간    1765
새벽     708
심야     578
Name: 시간구분, dtype: int64

In [31]:
test['계절']=test['월'].apply(seperate_month)
print(test['계절'].value_counts().sum())
test['계절'].value_counts()

10963


가을    2947
여름    2783
봄     2771
겨울    2462
Name: 계절, dtype: int64

In [32]:
test

,ID,요일,기상상태,노면상태,사고유형,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,대구광역시,수성구,상동,교차로,교차로안,2022,1,1,휴일,심야,겨울
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,대구광역시,수성구,지산동,단일로,기타,2022,1,1,휴일,심야,겨울
2,ACCIDENT_39611,토요일,맑음,건조,차대차,대구광역시,수성구,수성동2가,교차로,교차로안,2022,1,4,휴일,새벽,겨울
3,ACCIDENT_39612,토요일,맑음,건조,차대차,대구광역시,수성구,신매동,단일로,기타,2022,1,4,휴일,새벽,겨울
4,ACCIDENT_39613,토요일,맑음,건조,차대차,대구광역시,달서구,감삼동,교차로,교차로안,2022,1,6,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,토요일,맑음,건조,차대차,대구광역시,남구,대명동,단일로,터널,2022,12,18,휴일,저녁,겨울
10959,ACCIDENT_50568,토요일,맑음,건조,차대차,대구광역시,수성구,시지동,단일로,기타,2022,12,18,휴일,저녁,겨울
10960,ACCIDENT_50569,토요일,맑음,건조,차대차,대구광역시,수성구,연호동,단일로,기타,2022,12,20,휴일,야간,겨울
10961,ACCIDENT_50570,토요일,맑음,건조,차대차,대구광역시,수성구,범물동,교차로,교차로부근,2022,12,20,휴일,야간,겨울


#문자열로 된 숫자 정수형으로 변환

In [35]:
test['년도'] = test['년도'].astype(int)
test['월'] = test['월'].astype(int)
test['시간'] = test['시간'].astype(int)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   요일       10963 non-null  object
 1   기상상태     10963 non-null  object
 2   노면상태     10963 non-null  object
 3   사고유형     10963 non-null  object
 4   시        10963 non-null  object
 5   구        10963 non-null  object
 6   동        10963 non-null  object
 7   도로형태1    10963 non-null  object
 8   도로형태2    10963 non-null  object
 9   년도       10963 non-null  int64 
 10  월        10963 non-null  int64 
 11  시간       10963 non-null  int64 
 12  holiday  10963 non-null  object
 13  시간구분     10963 non-null  object
 14  계절       10963 non-null  object
dtypes: int64(3), object(12)
memory usage: 1.3+ MB


In [36]:
concatenated_df['년도'] = concatenated_df['년도'].astype(int)
concatenated_df['월'] = concatenated_df['월'].astype(int)
concatenated_df['시간'] = concatenated_df['시간'].astype(int)
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267353 entries, 0 to 602772
Data columns (total 20 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   요일       267353 non-null  object
 1   기상상태     267353 non-null  object
 2   노면상태     267353 non-null  object
 3   사고유형     267353 non-null  object
 4   사망자수     267353 non-null  int64 
 5   중상자수     267353 non-null  int64 
 6   경상자수     267353 non-null  int64 
 7   부상자수     267353 non-null  int64 
 8   ECLO     267353 non-null  int64 
 9   시        39609 non-null   object
 10  구        39609 non-null   object
 11  동        39609 non-null   object
 12  도로형태1    267353 non-null  object
 13  도로형태2    267353 non-null  object
 14  년도       267353 non-null  int64 
 15  월        267353 non-null  int64 
 16  시간       267353 non-null  int64 
 17  holiday  267353 non-null  object
 18  시간구분     267353 non-null  object
 19  계절       267353 non-null  object
dtypes: int64(8), object(12)
memory usage: 42.8+ MB


In [37]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타,2019,1,0,휴일,심야,겨울
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타,2019,1,0,휴일,심야,겨울
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타,2019,1,1,휴일,심야,겨울
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타,2019,1,2,휴일,심야,겨울
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타,2019,1,4,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,NaN,NaN,NaN,단일로,기타,2021,12,23,평일,야간,겨울
602769,금요일,맑음,건조,차대차,0,1,0,0,5,NaN,NaN,NaN,단일로,기타,2021,12,23,평일,야간,겨울
602770,금요일,흐림,건조,차대차,0,1,1,0,8,NaN,NaN,NaN,교차로,교차로안,2021,12,23,평일,야간,겨울
602771,금요일,맑음,건조,차대차,0,0,2,0,6,NaN,NaN,NaN,단일로,기타,2021,12,23,평일,야간,겨울


#원 핫 인코딩

train-set

In [38]:
total_data = pd.concat([concatenated_df, test])
total_data = pd.get_dummies(total_data)

In [39]:
for c in total_data.columns:
    print(c)

사망자수
중상자수
경상자수
부상자수
ECLO
년도
월
시간
요일_금요일
요일_목요일
요일_수요일
요일_월요일
요일_일요일
요일_토요일
요일_화요일
기상상태_기타
기상상태_눈
기상상태_맑음
기상상태_비
기상상태_안개
기상상태_흐림
노면상태_건조
노면상태_기타
노면상태_서리/결빙
노면상태_적설
노면상태_젖음/습기
노면상태_침수
노면상태_해빙
사고유형_차대사람
사고유형_차대차
사고유형_차량단독
시_대구광역시
구_남구
구_달서구
구_달성군
구_동구
구_북구
구_서구
구_수성구
구_중구
동_가창면
동_가천동
동_각산동
동_갈산동
동_감삼동
동_검단동
동_검사동
동_계산동1가
동_계산동2가
동_고모동
동_고성동1가
동_고성동2가
동_고성동3가
동_공평동
동_관음동
동_괴전동
동_교동
동_구암동
동_구지면
동_국우동
동_금강동
동_금호동
동_남산동
동_남성로
동_남일동
동_내당동
동_내동
동_노곡동
동_노변동
동_노원동1가
동_노원동2가
동_노원동3가
동_논공읍
동_능성동
동_다사읍
동_달성동
동_대곡동
동_대림동
동_대명동
동_대봉동
동_대신동
동_대안동
동_대천동
동_대현동
동_대흥동
동_덕곡동
동_덕산동
동_도남동
동_도동
동_도원동
동_도학동
동_동내동
동_동문동
동_동변동
동_동산동
동_동성로1가
동_동성로2가
동_동성로3가
동_동인동1가
동_동인동2가
동_동인동3가
동_동인동4가
동_동일동
동_동천동
동_동호동
동_두류동
동_두산동
동_둔산동
동_만촌동
동_매여동
동_매천동
동_매호동
동_문화동
동_미곡동
동_미대동
동_방촌동
동_백안동
동_범물동
동_범어동
동_복현동
동_본동
동_본리동
동_봉덕동
동_봉무동
동_봉산동
동_부동
동_북성로1가
동_북성로2가
동_불로동
동_비산동
동_사복동
동_사수동
동_사월동
동_사일동
동_산격동
동_삼덕동
동_삼덕동1가
동_삼덕동2가
동_삼덕동3가
동_상동
동_상리동
동_상매동
동_상서동
동_상인동
동_서문로1가
동_서문로2가
동_서변동
동_서성로1가
동_서성로2가
동_서야동
동_서호동
동_성당동
동_성동
동_송정동
동_송현동

In [40]:
train_data = total_data[~total_data['ECLO'].isnull()]
test_data = total_data[total_data['ECLO'].isnull()]

In [41]:
test_data = test_data.drop(['사망자수','중상자수','경상자수','부상자수','ECLO'], axis=1)

In [42]:
train_data

,사망자수,중상자수,경상자수,부상자수,ECLO,년도,월,시간,요일_금요일,요일_목요일,...,시간구분_새벽,시간구분_심야,시간구분_야간,시간구분_오전,시간구분_오후,시간구분_저녁,계절_가을,계절_겨울,계절_봄,계절_여름
0,0.0,1.0,0.0,0.0,5.0,2019,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,0.0,0.0,1.0,0.0,3.0,2019,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,0.0,0.0,1.0,0.0,3.0,2019,1,1,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0.0,1.0,0.0,0.0,5.0,2019,1,2,0,0,...,0,1,0,0,0,0,0,1,0,0
4,0.0,0.0,1.0,0.0,3.0,2019,1,4,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,0.0,0.0,1.0,0.0,3.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0
602769,0.0,1.0,0.0,0.0,5.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0
602770,0.0,1.0,1.0,0.0,8.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0
602771,0.0,0.0,2.0,0.0,6.0,2021,12,23,1,0,...,0,0,1,0,0,0,0,1,0,0


In [43]:
test_data

,년도,월,시간,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,시간구분_새벽,시간구분_심야,시간구분_야간,시간구분_오전,시간구분_오후,시간구분_저녁,계절_가을,계절_겨울,계절_봄,계절_여름
0,2022,1,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
1,2022,1,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,2022,1,4,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
3,2022,1,4,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,2022,1,6,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,2022,12,18,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
10959,2022,12,18,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
10960,2022,12,20,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
10961,2022,12,20,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0


In [45]:
from scipy import stats

# 샤피로-윌크 검정
stat, p_value = stats.shapiro(concatenated_df['ECLO'])
print(f"Shapiro-Wilk test: Statistic={stat}, p-value={p_value}")

Shapiro-Wilk test: Statistic=0.6575490236282349, p-value=0.0


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [55]:
# 인코딩할 열 선택
string_columns = concatenated_df.columns[1:]

# get_dummies를 사용해 원핫 인코딩
string_encoded = pd.get_dummies(concatenated_df, columns=string_columns)

# boolean 값을 0과 1로 변환
string_encoded = string_encoded.iloc[:, 1: ].astype(float)

# 변환된 결과 출력
display(string_encoded.head())
print(string_encoded.columns.to_list())

,기상상태_기타,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_안개,기상상태_흐림,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,...,시간구분_새벽,시간구분_심야,시간구분_야간,시간구분_오전,시간구분_오후,시간구분_저녁,계절_가을,계절_겨울,계절_봄,계절_여름
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


['기상상태_기타', '기상상태_눈', '기상상태_맑음', '기상상태_비', '기상상태_안개', '기상상태_흐림', '노면상태_건조', '노면상태_기타', '노면상태_서리/결빙', '노면상태_적설', '노면상태_젖음/습기', '노면상태_침수', '노면상태_해빙', '사고유형_차대사람', '사고유형_차대차', '사고유형_차량단독', '사망자수_0', '사망자수_1', '사망자수_2', '사망자수_3', '사망자수_5', '중상자수_0', '중상자수_1', '중상자수_2', '중상자수_3', '중상자수_4', '중상자수_5', '중상자수_6', '중상자수_7', '중상자수_8', '중상자수_9', '중상자수_10', '중상자수_11', '중상자수_12', '경상자수_0', '경상자수_1', '경상자수_2', '경상자수_3', '경상자수_4', '경상자수_5', '경상자수_6', '경상자수_7', '경상자수_8', '경상자수_9', '경상자수_10', '경상자수_11', '경상자수_12', '경상자수_13', '경상자수_14', '경상자수_15', '경상자수_16', '경상자수_17', '경상자수_18', '경상자수_19', '경상자수_20', '경상자수_22', '경상자수_27', '경상자수_28', '경상자수_38', '경상자수_41', '부상자수_0', '부상자수_1', '부상자수_2', '부상자수_3', '부상자수_4', '부상자수_5', '부상자수_6', '부상자수_7', '부상자수_8', '부상자수_9', '부상자수_10', '부상자수_13', '부상자수_16', '부상자수_17', '부상자수_25', '부상자수_47', 'ECLO_1', 'ECLO_2', 'ECLO_3', 'ECLO_4', 'ECLO_5', 'ECLO_6', 'ECLO_7', 'ECLO_8', 'ECLO_9', 'ECLO_10', 'ECLO_11', 'ECLO_12', 'ECLO_13', 'ECLO_14', 'ECLO_15', 'ECLO_16', 'ECLO_17', 'ECLO_18', 

In [57]:
train_drop_string  = string_encoded[:len(concatenated_df)]
test_drop_string = string_encoded[len(concatenated_df):]

In [58]:
train_drop_string.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267353 entries, 0 to 602772
Columns: 427 entries, 기상상태_기타 to 계절_여름
dtypes: float64(427)
memory usage: 873.0 MB


In [59]:
test_drop_string.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Columns: 427 entries, 기상상태_기타 to 계절_여름
dtypes: float64(427)
memory usage: 0.0 bytes


In [60]:
import lightgbm as lgb          #LightGBM: XGBoost와 유사하며, 대규모 데이터셋 처리에 특화되어 있어 학습 속도가 매우 빠릅니다.
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

concatenated_df_column = 'ECLO'         #훈련 데이터에서 target ECLO 추출
eclo_y_train = concatenated_df[concatenated_df_column]

cd_features = train_drop_string         #훈련 데이터에는 필요없을 것 같아서 열 제거
X_train, X_val, y_train, y_val = train_test_split(cd_features, eclo_y_train, test_size = 0.3)

In [61]:
print("Size of X_train is:{}\nSize of y_train is:{}\nSize of X_valid is:{}\nSize of y_valid is:{}\n".format(
    X_train.shape,y_train.shape,X_val.shape,y_val.shape))

Size of X_train is:(187147, 427)
Size of y_train is:(187147,)
Size of X_valid is:(80206, 427)
Size of y_valid is:(80206,)



In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

rf_reg = RandomForestRegressor()

In [63]:
rf_reg.fit(X_train, y_train)
rf_pred = rf_reg.predict(X_val)

In [64]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

#  1. RMSLE 작성
def rmsle(pred, actual):
    log_pred = np.log1p(pred)
    log_actual = np.log1p(actual)
    squared_error = (log_pred-log_actual)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle
#  2. RMSE 작성
def rmse(pred, actual):
    return np.sqrt(mean_squared_error(pred, actual))
#  3. MAE 작성
def mae(pred, actual):
    return mean_absolute_error(pred, actual)

def evaluate_regr(pred, actual):
    rmsle_val=rmsle(pred, actual)
    rmsl_val=rmse(pred, actual)
    mae_val=mae(pred, actual)
    print('RMSLE: {:.3f}, RMSE: {:.3f}, MAE: {:.3f}'.format(rmsle_val, rmsl_val, mae_val))

In [65]:
evaluate_regr(rf_pred, y_val)

RMSLE: 0.008, RMSE: 0.508, MAE: 0.007
